In [88]:
from dataclasses import dataclass, field
from typing import List
from pydantic import BaseModel, Field

from langchain_core.messages import AIMessage, SystemMessage, HumanMessage
from langgraph.graph import StateGraph, START, END, MessagesState
from langchain.tools import tool 
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition
from langchain_openai import ChatOpenAI

from dotenv import load_dotenv

load_dotenv()

True

In [89]:
system_recommender_prompt = """
    You are a book expert.
    Your job is recommend books.
    Recommend some books in bases in the user request.
    If the user doesn't request an specific number of books, recomend 3.
    If the user request more than 5 books, recommend only 5 and tell the user that 
    you are only allow to recommend a maximun of 5 books.
    If the user doesn't ask for book recommendations or something similar, tell 
    the user that you are not capable of helping him.

    previous books recommended:
    {previous_books}

    This is the user query:
    
"""

In [116]:
class Book(BaseModel):
    """Book information"""
    name: str = Field(description="The name of the book")
    author: str = Field(description="The book's author name")

    def __str__(self):
        return f"{self.name} — {self.author}"
        
@tool
class RecommendedBooks(BaseModel):
    """A list of recommended books"""
    recommended_books: List[Book] = Field(description="The different recommended books")

    def __str__(self):
        if recommended_books:
            return "\n".join(str(book) for book in self.recomended_books)
        else:
            return ""

In [125]:
@dataclass
class RecState(MessagesState):
    recommended_books: List[Book] = field(default_factory=list)

def thinking_node(state: RecState):
    print("Executing thinking node")

    chain = ChatOpenAI(model="gpt-3.5-turbo")
    prompt = system_recommender_prompt.format(previous_books=str(state.get("recommended_books", [])))
    
    result = chain.invoke([SystemMessage(content=system_recommender_prompt), 
                           state["messages"][-1]])

    return {"messages": AIMessage(content=result.content)}

def save_recommended_books(state: RecState):
    print("Saving Recommended Books")

    model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
    
    model_with_structure  = model.with_structured_output(RecommendedBooks)

    recommended_books = model_with_structure.invoke([state["messages"][-1]])

    if recommended_books.get("recommended_books", False):
        return {"recommended_books": recommended_books["recommended_books"]}
    

In [126]:
builder = StateGraph(RecState)
builder.add_node("thinking", thinking_node)
builder.add_node("save_recommended_books", save_recommended_books)

builder.add_edge(START, "thinking")
builder.add_edge("thinking", "save_recommended_books")
builder.add_edge("save_recommended_books", END)

graph = builder.compile()

In [127]:
mensaje_inicial = "Recommend me some books about individualism"

In [128]:
graph.invoke({"messages": [HumanMessage(content=mensaje_inicial)]})

Executing thinking node
Saving Recommended Books


C:\Users\Erik\Desktop\Book-recommender-AI-agent\.venv\Lib\site-packages\langchain_openai\chat_models\base.py:1857: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


{'messages': [HumanMessage(content='Recommend me some books about individualism', additional_kwargs={}, response_metadata={}, id='b38b3e8c-c6aa-4393-8dcd-d1e779f734b8'),
  AIMessage(content="I'm sorry, but I am not capable of recommending specific books about individualism.", additional_kwargs={}, response_metadata={}, id='ea59a9b0-ed69-40a8-8bac-51ee64330eb2')],
 'recommended_books': [{'name': 'Atlas Shrugged', 'author': 'Ayn Rand'},
  {'name': 'The Fountainhead', 'author': 'Ayn Rand'},
  {'name': 'Anthem', 'author': 'Ayn Rand'}]}

In [100]:
mensaje_erroneo = "Who is the current FC Barcelona manager?"

graph.invoke({"messages": [HumanMessage(content=mensaje_erroneo)],
              "recommended_books": []})

Executing thinking node
Saving Recommended Books


C:\Users\Erik\Desktop\Book-recommender-AI-agent\.venv\Lib\site-packages\langchain_openai\chat_models\base.py:1857: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


{'messages': [HumanMessage(content='Who is the current FC Barcelona manager?', additional_kwargs={}, response_metadata={}, id='b8a62f18-24a0-4e75-8932-f3f6997df720'),
  AIMessage(content="I'm sorry, I am not capable of helping with that question. If you would like book recommendations, please let me know the genre or topic you are interested in.", additional_kwargs={}, response_metadata={}, id='b0bd5978-e11c-4571-b0d6-753654cb52f7')],
 'recommended_books': []}